In [7]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Constants
IMG_HEIGHT, IMG_WIDTH = 128, 128
BATCH_SIZE = 32
ORGANIZED_DIR = "organized_dataset"

# Data augmentation and preprocessing
datagen = ImageDataGenerator(
    rescale=1.0/255,  # Normalize pixel values
    validation_split=0.2  # Split training and validation data
)

# Training data
train_data = datagen.flow_from_directory(
    ORGANIZED_DIR,
    target_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='training'
)

# Validation data
val_data = datagen.flow_from_directory(
    ORGANIZED_DIR,
    target_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='validation'
)


Found 44875 images belonging to 6 classes.
Found 11217 images belonging to 6 classes.


In [8]:
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Dropout
from tensorflow.keras.optimizers import Adam

# Load MobileNetV2
base_model = MobileNetV2(weights="imagenet", include_top=False, input_shape=(IMG_HEIGHT, IMG_WIDTH, 3))
base_model.trainable = False  # Freeze pre-trained layers

# Add custom layers
model = Sequential([
    base_model,
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(train_data.num_classes, activation='softmax')  # Number of classes = 6
])

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.001), loss="categorical_crossentropy", metrics=["accuracy"])

# Summary
model.summary()


9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 12s 1us/step


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ mobilenetv2_1.00_128            │ (None, 4, 4, 1280)     │     2,257,984 │
│ (Functional)                    │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 20480)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │     2,621,568 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 6)              │           774 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 4,880,326 (18.62 MB)

 Trainable params: 2,622,342 (10.00 MB)

 Non-trainable params: 2,257,984 (8.61 MB)

In [9]:
# Train the model
history = model.fit(
    train_data,
    validation_data=val_data,
    epochs=10  # Adjust based on dataset size
)

# Save the model
model.save("road_condition_detector.h5")


d:\Machine Learning FYP\machine\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/10
1403/1403 ━━━━━━━━━━━━━━━━━━━━ 668s 474ms/step - accuracy: 0.6835 - loss: 1.1250 - val_accuracy: 0.8162 - val_loss: 0.5191
Epoch 2/10
1403/1403 ━━━━━━━━━━━━━━━━━━━━ 526s 375ms/step - accuracy: 0.7683 - loss: 0.6401 - val_accuracy: 0.8545 - val_loss: 0.4593
Epoch 3/10
1403/1403 ━━━━━━━━━━━━━━━━━━━━ 514s 366ms/step - accuracy: 0.7980 - loss: 0.5790 - val_accuracy: 0.8614 - val_loss: 0.4361
Epoch 4/10
1403/1403 ━━━━━━━━━━━━━━━━━━━━ 552s 393ms/step - accuracy: 0.8142 - loss: 0.5397 - val_accuracy: 0.8783 - val_loss: 0.4125
Epoch 5/10
1403/1403 ━━━━━━━━━━━━━━━━━━━━ 571s 407ms/step - accuracy: 0.8187 - loss: 0.5120 - val_accuracy: 0.8657 - val_loss: 0.4785
Epoch 6/10
1403/1403 ━━━━━━━━━━━━━━━━━━━━ 415s 296ms/step - accuracy: 0.8237 - loss: 0.5057 - val_accuracy: 0.8715 - val_loss: 0.4317
Epoch 7/10
1403/1403 ━━━━━━━━━━━━━━━━━━━━ 519s 370ms/step - accuracy: 0.8293 - loss: 0.4929 - val_accuracy: 0.8709 - val_loss: 0.4235
Epoch 8/10
1403/1403 ━━━━━━━━━━━━━━━━━━━━ 744s 530ms/step - ac

In [10]:
import cv2
import numpy as np
from tensorflow.keras.models import load_model

# Load the trained model
model = load_model("road_condition_detector.h5")

# Class labels
class_labels = list(train_data.class_indices.keys())

# Start webcam feed
cap = cv2.VideoCapture(0)

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Preprocess frame
    resized_frame = cv2.resize(frame, (IMG_HEIGHT, IMG_WIDTH))
    img = np.expand_dims(resized_frame, axis=0) / 255.0

    # Predict
    prediction = model.predict(img)
    predicted_class = class_labels[np.argmax(prediction)]

    # Display on frame
    cv2.putText(frame, f"Condition: {predicted_class}", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
    cv2.imshow("Road Condition Detector", frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 54ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
1/1 ━━━━━━━━━━